In [180]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from tensorflow import keras
from sklearn.metrics import mean_absolute_error
from datetime import datetime

In [181]:
#                        .::::.
#                      .::::::::.
#                     :::::::::::
#                  ..:::::::::::'
#               '::::::::::::'
#                 .::::::::::
#            '::::::::::::::..
#                 ..::::::::::::.
#               ``::::::::::::::::
#                ::::``:::::::::'        .:::.
#               ::::'   ':::::'       .::::::::.
#             .::::'      ::::     .:::::::'::::.
#            .:::'       :::::  .:::::::::' ':::::.
#           .::'        :::::.:::::::::'      ':::::.
#          .::'         ::::::::::::::'         ``::::.
#      ...:::           ::::::::::::'              ``::.
#     ```` ':.          ':::::::::'                  ::::..
#                        '.:::::'                    ':'````..
#                     美女保佑 永无BUG

In [182]:
data_train = pd.read_csv(r'./data/used_car_train_20200313.csv',sep = ' ')
data_test = pd.read_csv(r'./data/used_car_testB_20200421.csv',sep = ' ')

# 参考代码

In [133]:
data_train.replace(to_replace = '-', value = np.nan, inplace = True)
data_test.replace(to_replace = '-', value = np.nan, inplace = True)

data_train.fillna(data_train.median(),inplace= True)
data_test.fillna(data_train.median(),inplace= True)

#修改异常数据
data_train['power'][data_train['power']>600]=600
data_test['power'][data_test['power']>600]=600

#特征标签
tags = ['model','brand','bodyType','fuelType','regionCode','regionCode','regDate','creatDate','kilometer','notRepairedDamage','power','v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
       'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14']

#特征归一化
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(data_train[tags].values)
x = min_max_scaler.transform(data_train[tags].values)
x_ = min_max_scaler.transform(data_test[tags].values)

#获得y值
y = data_train['price'].values

d:\python\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\python\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


# 原baseline

In [183]:
train = data_train.copy()
test = data_test.copy()

In [184]:
# 转换'-'
train['notRepairedDamage'] = train['notRepairedDamage'].replace('-', np.nan) 
test['notRepairedDamage'] = test['notRepairedDamage'].replace('-', np.nan)

# 转换数据类型
train['notRepairedDamage'] = train['notRepairedDamage'].astype('float64')
test['notRepairedDamage'] = test['notRepairedDamage'].astype('float64')


In [185]:
def date_process(x):
    year = int(str(x)[:4])
    month = int(str(x)[4:6])
    day = int(str(x)[6:8])
    if month < 1:
        date = str(x)[:4] + "07" + str(x)[6:]
    else:
        date = str(x)
    return date

In [186]:
# apply()是将自定义函数作用到Series或者DataFrame中
train['creatDate'] = train['creatDate'].apply(date_process)
train['regDate'] = train['regDate'].apply(date_process)
test['creatDate'] = test['creatDate'].apply(date_process)
test['regDate'] = test['regDate'].apply(date_process)

In [187]:
train.drop(['seller'], axis=1, inplace=True)
test.drop(['seller'], axis=1, inplace=True)

train = train.drop(['offerType'], axis=1)
test = test.drop(['offerType'], axis=1)

In [188]:
# 使用map函数，以power列的中位数来替换数值超出范围的power
train['power'] = train['power'].map(lambda x: train['power'].mean() if x > 600 else x)
test['power'] = test['power'].map(lambda x: test['power'].mean() if x > 600 else x)

In [189]:
# 填补缺失值
train.loc[38424, 'model'] = 157.0

# 在两个数据集上，车身类型为0.0（豪华轿车）的汽车数量都是最多，所以用0.0来填充缺失值
train.loc[:, 'bodyType'] = train['bodyType'].map(lambda x: 0.0 if pd.isnull(x) else x)
test.loc[:, 'bodyType'] = test['bodyType'].map(lambda x: 0.0 if pd.isnull(x) else x)


# 猜想：燃油类型与车身类型相关，如豪华轿车更可能是汽油或电动， 而搅拌车大多是柴油
# 创建字典map，保存不同bodyType下， fuelType的众数，并以此填充fuelTyp的缺失值
dict_enu_train, dict_enu_test = {}, {}
for i in [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]:
    dict_enu_train[i] = train[train['bodyType'] == i]['fuelType'].mode()[0] # bodyType=i时，fuelType的众数
    dict_enu_test[i] = test[test['bodyType'] == i]['fuelType'].mode()[0]
# 发现dict_enu_train, dict_enu_test是一样的内容
# 开始填充fuelType缺失值
# 在含fuelType缺失值的条目中，将不同bodyType对应的index输出保存到一个字典中
dict_index_train, dict_index_test = {}, {}
for bodytype in [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]:
    dict_index_train[bodytype] = train[(train['bodyType'] == bodytype) & (train['fuelType'].isnull())].index.tolist()
    dict_index_test[bodytype] = test[(test['bodyType'] == bodytype) & (test['fuelType'].isnull())].index.tolist()
# 分别对每个bodyTYpe所对应的index来填充fuelType列
for bt, ft in dict_enu_train.items():
#     train.loc[tuple(dict_index[bt]), :]['fuelType'] = ft  # 注意：链式索引 (chained indexing)很可能导致赋值失败！
    train.loc[dict_index_train[bt], 'fuelType'] = ft  # Pandas推荐使用这种方法来索引/赋值
    test.loc[dict_index_test[bt], 'fuelType'] = ft
    

# 训练集，如果缺失，补0
train.loc[:, 'gearbox'] = train['gearbox'].map(lambda x: 0.0 if pd.isnull(x) else x)
# # 对于测试集，为保证预测结果完整性，不能删去任何行。测试集仅有1910个gearbox缺失值，用数量占绝大多数的0.0（手动档）来填充缺失值
test.loc[:, 'gearbox'] = test['gearbox'].map(lambda x: 0.0 if pd.isnull(x) else x)


# 在整个训练集上有尚未修复损坏的汽车比损坏已修复的汽车售价还要高。
# 为简单化问题，仍使用数量占比最大的0.0来填充所有缺失值
train.loc[:, 'notRepairedDamage'] = train['notRepairedDamage'].map(lambda x: 0.0 if pd.isnull(x) else x)
test.loc[:, 'notRepairedDamage'] = test['notRepairedDamage'].map(lambda x: 0.0 if pd.isnull(x) else x)

In [190]:
# #特征标签
# tags = ['model','brand','bodyType','fuelType','regionCode','regionCode','regDate','creatDate','kilometer','notRepairedDamage','power','v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
#        'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14']

# #特征归一化
# min_max_scaler = MinMaxScaler()
# min_max_scaler.fit(train[tags].values)
# x = min_max_scaler.transform(train[tags].values)
# x_ = min_max_scaler.transform(test[tags].values)

# #获得y值
# y = train['price'].values

In [191]:
#获得y值
y = train['price'].values
del train['price']

# 特征归一化
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(train.values)
x = min_max_scaler.transform(train.values)
x_test = min_max_scaler.transform(test.values)

# 建模

In [192]:
#切分数据集
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size = 0.15, random_state = 22)

In [193]:
model = keras.Sequential([
        keras.layers.Dense(250,activation='relu',input_shape=[28]), 
        keras.layers.Dense(250,activation='relu'), 
        keras.layers.Dense(250,activation='relu'), 
        keras.layers.Dense(1)])
model.compile(loss='mean_absolute_error',
                optimizer='Adam')

In [194]:
model.fit(x_train,y_train,batch_size = 256, epochs = 200)

Epoch 1/200
499/499 [==============================] - 1s 2ms/step - loss: 2455.8481
Epoch 2/200
499/499 [==============================] - 1s 2ms/step - loss: 986.3889
Epoch 3/200
499/499 [==============================] - 1s 2ms/step - loss: 877.4416
Epoch 4/200
499/499 [==============================] - 1s 2ms/step - loss: 815.8169
Epoch 5/200
499/499 [==============================] - 1s 3ms/step - loss: 765.5081
Epoch 6/200
499/499 [==============================] - 1s 2ms/step - loss: 730.4852
Epoch 7/200
499/499 [==============================] - 1s 2ms/step - loss: 706.6591
Epoch 8/200
499/499 [==============================] - 2s 3ms/step - loss: 683.4187
Epoch 9/200
499/499 [==============================] - 1s 3ms/step - loss: 668.3130
Epoch 10/200
499/499 [==============================] - 1s 3ms/step - loss: 650.6030
Epoch 11/200
499/499 [==============================] - 1s 3ms/step - loss: 637.9131
Epoch 12/200
499/499 [==============================] - 1s 3ms/step - los

499/499 [==============================] - 2s 3ms/step - loss: 490.9262
Epoch 98/200
499/499 [==============================] - 2s 3ms/step - loss: 488.5401
Epoch 99/200
499/499 [==============================] - 2s 3ms/step - loss: 489.3480
Epoch 100/200
499/499 [==============================] - 2s 3ms/step - loss: 484.9375
Epoch 101/200
499/499 [==============================] - 2s 3ms/step - loss: 486.8741
Epoch 102/200
499/499 [==============================] - 2s 3ms/step - loss: 486.8351
Epoch 103/200
499/499 [==============================] - 2s 3ms/step - loss: 484.4384
Epoch 104/200
499/499 [==============================] - 2s 3ms/step - loss: 486.7724
Epoch 105/200
499/499 [==============================] - 2s 3ms/step - loss: 483.4712
Epoch 106/200
499/499 [==============================] - 2s 3ms/step - loss: 485.0384
Epoch 107/200
499/499 [==============================] - 1s 3ms/step - loss: 483.3489
Epoch 108/200
499/499 [==============================] - 2s 3ms/step -

499/499 [==============================] - 1s 3ms/step - loss: 458.4982
Epoch 192/200
499/499 [==============================] - 2s 3ms/step - loss: 463.0638
Epoch 193/200
499/499 [==============================] - 2s 3ms/step - loss: 462.1920
Epoch 194/200
499/499 [==============================] - 2s 3ms/step - loss: 461.2415
Epoch 195/200
499/499 [==============================] - 2s 3ms/step - loss: 463.1223A: 0s - loss: 461.6
Epoch 196/200
499/499 [==============================] - 2s 3ms/step - loss: 460.8319
Epoch 197/200
499/499 [==============================] - 2s 3ms/step - loss: 462.7691
Epoch 198/200
499/499 [==============================] - 2s 3ms/step - loss: 462.2745
Epoch 199/200
499/499 [==============================] - 2s 3ms/step - loss: 460.0641
Epoch 200/200
499/499 [==============================] - 2s 3ms/step - loss: 458.4085


In [195]:
#比较训练集和测试集效果
nn_train_pre = model.predict(x_train)
pd.DataFrame(nn_train_pre).to_csv('./data/nn_train_pre.csv', sep = ',', index = None)
print(mean_absolute_error(y_train, nn_train_pre))

470.5117184837154


In [196]:
nn_val_pre = model.predict(x_val)
pd.DataFrame(nn_val_pre).to_csv('./data/nn_val_pre.csv', sep = ',', index = None)
print(mean_absolute_error(y_val, nn_val_pre))

478.9399366072761


In [197]:
#输出结果预测
nn_test_pre = model.predict(x_test)
pd.DataFrame(nn_test_pre).to_csv('./data/nn_test_pre.csv', sep = ',', index = None)

In [198]:
sub= pd.read_csv(r'./data/used_car_sample_submit.csv', sep=',')
sub['price'] = pd.Series(np.squeeze(nn_test_pre))
sub.to_csv('./data/sub_network.csv',sep = ',',index = None)